In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('C:/Users/Tony/Desktop/orginal.csv',parse_dates = ['T'])[['T','AT']]

In [3]:
training_data_len=int(np.ceil( len(data['AT']) * .95 )) #get the 95% of rows
training_data_len

1726

In [4]:
AT=np.array(data['AT'])
temp_values=AT.reshape(AT.shape[0],1)

In [5]:
from sklearn.preprocessing import MinMaxScaler #preprocessing
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(temp_values)

scaled_data

array([[0.89885103],
       [0.91704855],
       [0.90961015],
       ...,
       [0.96387062],
       [0.8364216 ],
       [0.68101215]])

In [6]:
# Create the training data set 
# Create the scaled training data set
train_data = scaled_data[0:int(training_data_len), :]
# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
        
# Convert the x_train and y_train to numpy arrays 
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# x_train.shape

[array([0.89885103, 0.91704855, 0.90961015, 0.92687786, 0.82353723,
       0.87600452, 0.85481836, 0.70173341, 0.91027429, 0.88138407,
       0.84027363, 0.51205419, 0.84439131, 0.88629873, 0.84744637,
       0.70744504, 0.55137146, 0.53802218, 0.8499037 , 0.70392508,
       0.56100153, 0.85070067, 0.82971375, 0.88231387, 0.68718868,
       0.69861194, 0.71036727, 0.83960948, 0.6891147 , 0.57282327,
       0.81981802, 0.66600252, 0.8554825 , 0.75599389, 0.84937238,
       0.93564455, 0.88324367, 0.72976024, 0.78601315, 0.87746563,
       0.93385136, 0.8615262 , 0.75991233, 0.51796507, 0.83416351,
       0.88576742, 0.73208474, 0.56325961, 0.82978017, 0.91133692,
       0.88118483, 0.74895397, 0.56392376, 0.68971243, 0.85109916,
       0.91053995, 0.87846184, 0.72717009, 0.56936973, 0.23636847])]
[0.5031546788868964]

[array([0.89885103, 0.91704855, 0.90961015, 0.92687786, 0.82353723,
       0.87600452, 0.85481836, 0.70173341, 0.91027429, 0.88138407,
       0.84027363, 0.51205419, 0.844

In [9]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, batch_size=50, epochs=5)

Epoch 1/5
34/34 [==============================] - 2s 13ms/step - loss: 0.1010
Epoch 2/5
34/34 [==============================] - 0s 8ms/step - loss: 0.0565
Epoch 3/5
34/34 [==============================] - 0s 8ms/step - loss: 0.0084
Epoch 4/5
34/34 [==============================] - 0s 8ms/step - loss: 0.0045
Epoch 5/5
34/34 [==============================] - 0s 8ms/step - loss: 0.0037


In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 128)           66560     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 25)                1625      
                                                                 
 dense_5 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Create the testing data set
# Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = temp_values[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Get the models predicted price values 
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
rmse

In [ ]:
temperature=data['AT']
train = temperature[:training_data_len]
valid=pd.DataFrame(columns=['AvgTemperature', 'Predictions'])
valid['AvgTemperature'] = temperature[training_data_len:]
valid['Predictions'] = predictions
# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('AvgTemperature', fontsize=18)
plt.plot(train)
plt.plot(valid[['AvgTemperature', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
# plt.show()
plt.savefig('plot1_2_2_month.png', dpi=300) #指定分辨率保存